# Installation Notes for K8s and KF with JuJu on GCP VM

## Instructions: 
- https://www.kubeflow.org/docs/distributions/charmed/install-kubeflow/
- https://charmed-kubeflow.io/docs/install

## First Config
```
# Create VM with Ubuntu 20.04, 8vCPU, 32GiB, 120GB HDD


# Connect with SSH from GCP console (t1)
snap list
sudo snap install microk8s --classic
sudo snap install juju --classic
snap list

microk8s status --wait-ready
sudo usermod -a -G microk8s krzysztof_satola
sudo chown -f -R krzysztof_satola ~/.kube
exit


# Connect (again) with SSH from GCP console (t1)
microk8s status --wait-ready
kubectl get pods --all-namespaces #error

alias kubectl='microk8s kubectl'
kubectl get pods --all-namespaces
microk8s enable dns storage


# Wait for a while
kubectl get po -A


# Should return errors at first
juju models
juju controllers
juju add-k8s myk8s

microk8s config
microk8s config > ~/.kube/config
juju add-k8s myk8s

juju clouds
juju bootstrap myk8s my-controller
juju controllers
kubectl get po -A
kubectl get namespaces

juju add-model kubeflow
juju models
kubectl get namespaces
juju status


# Open a new SSH terminal session (t2)
watch --help
watch -n 1 juju status --color


# Switch to t1
juju deploy kubeflow-lite
#watch -c kubectl get po -n kubeflow # alias may not work
#watch -c microk8s kubectl get po -A
watch -c microk8s kubectl get po -n kubeflow
^+Z

kubectl patch role -n kubeflow istio-ingressgateway-operator -p '{"apiVersion":"rbac.authorization.k8s.io/v1","kind":"Role","metadata":{"name":"istio-ingressgateway-operator"},"rules":[{"apiGroups":["*"],"resources":["*"],"verbs":["*"]}]}'

#watch -c kubectl get po -n kubeflow
watch -c microk8s kubectl get po -n kubeflow

juju deploy argo-ui
juju relate argo-ui istio-pilot
juju remove-relation argo-ui istio-pilot
juju remove-application argo-ui


# https://charmed-kubeflow.io/docs/dashboard
juju config dex-auth static-username
juju config dex-auth static-password

juju config dex-auth static-username=admin
juju config dex-auth static-password=adminpass


kubectl get services -n kubeflow

# Connect to Kubeflow Dashboard
# https://charmed-kubeflow.io/docs/dashboard
# kubeflow-dashboard: 10.152.183.238 - 8082/TCP
# if this setup is run locally: http://10.152.183.238.nip.io
# if remotely:
# 1. Logout from the current session with the exit command
# 2. Re-establish connection to the machine using ssh with SOCKS proxy enabled through the -D 9999 parameter
# 3. On your computer, go to Settings > Network > Network Proxy, and enable SOCKS proxy pointing to: 127.0.0.1:9999
# 4. On a new browser window, access the link given in the previous step, appended by .nip.io, for example: http://10.152.183.238.nip.io


# Generate ssh keys to connect to GCP VM
# In the local terminal (t3)
ssh-keygen -t rsa -f ~/.ssh/gcp-ssh-key -C <E-MAIL used in GCP>
chmod 600 ~/.ssh/gcp-ssh-key

cat ~/.ssh/gcp-ssh-key.pub
# copy the full content of the .pub certificate

# In t1
cd ~/.ssh/
echo "<PASTE the .pub content file>" > authorized_keys
chmod 600 authorized_keys
# check if the file exists
ls -lha


# Check if you can connect from t3 to GCP VM
# https://askubuntu.com/questions/311558/ssh-permission-denied-publickey
ssh -i ~/.ssh/gcp-ssh-key krzysztof_satola@34.140.144.179


# Setup SOCKS proxy in Settings > Network > Network Proxy
# 127.0.0.1:9999
# 9999 is defined as -D in ssh below

# Run in t3 (locally)
ssh -i ~/.ssh/gcp-ssh-key -D 9999 krzysztof_satola@34.140.144.179

# And open in a local web browser (but this is not working properly, see KF subpages)
http://10.152.183.238.nip.io:8082/

# https://microk8s.io/docs/addon-metallb
microk8s enable metallb:10.64.140.43-10.64.140.49

# In t1
kubectl get svc -A | grep istio

# https://charmed-kubeflow.io/docs/install
# load balancer ip
juju config dex-auth public-url=http://10.152.183.160.nip.io
juju config oidc-gatekeeper public-url=http://10.152.183.160.nip.io
```

## After VM Restart
```
# Restart GCP VM

# Remote terminal (t1)
alias kubectl='microk8s kubectl'
watch -c microk8s kubectl get po -n kubeflow

# Locally (t2)
ssh -i ~/.ssh/gcp-ssh-key -D 9999 krzysztof_satola@34.140.144.179

# Turn on SOCKS proxy in Settings > Network > Network Proxy
# 127.0.0.1:9999
# 9999 is defined as -D in ssh above

# Open in a web browser
http://10.152.183.160.nip.io
admin
adminpass
```